<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork23455645-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# Peer Review Assignment - Data Engineer - ETL


Estimated time needed: **20** minutes


## Objectives

In this final part you will:

*   Run the ETL process
*   Extract bank and market cap data from the JSON file `bank_market_cap.json`
*   Transform the market cap currency using the exchange rate data
*   Load the transformed data into a seperate CSV


For this lab, we are going to be using Python and several Python libraries. Some of these libraries might be installed in your lab environment or in SN Labs. Others may need to be installed by you. The cells below will install these libraries when executed.


In [1]:
#!mamba install pandas==1.3.3 -y
#!mamba install requests==2.26.0 -y

## Imports

Import any additional libraries you may need here.


In [2]:
import glob
import pandas as pd
from pandas import json_normalize
from datetime import datetime

As the exchange rate fluctuates, we will download the same dataset to make marking simpler. This will be in the same format as the dataset you used in the last section


In [3]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv

--2023-03-03 14:54:56--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2815 (2.7K) [application/json]
Saving to: ‘bank_market_cap_1.json’

bank_market_cap_1.j 100%[===================>]   2.75K  --.-KB/s    in 0s      

2023-03-03 14:54:56 (83.7 MB/s) - ‘bank_market_cap_1.json’ saved [2815/2815]

--2023-03-03 14:54:57--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Tra

## Extract


### JSON Extract Function

This function will extract JSON files.


In [4]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe
df=extract_from_json('bank_market_cap_1.json')


## Extract Function

Define the extract function that finds JSON file `bank_market_cap_1.json` and calls the function created above to extract data from them. Store the data in a `pandas` dataframe. Use the following list for the columns.


In [365]:
columns=['Name','Market Cap (US$ Billion)']

In [5]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe
df=extract_from_json('bank_market_cap_1.json')
df=pd.DataFrame(df,columns=['Name','Market Cap (US$ Billion)'])
pd.set_option('display.max_columns', None)
df.style

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934000
1,Industrial and Commercial Bank of China,345.214000
2,Bank of America,325.331000
3,Wells Fargo,308.013000
4,China Construction Bank,257.399000
5,HSBC Holdings PLC,219.270000
6,Agricultural Bank of China,203.244000
7,Citigroup Inc.,203.165000
8,Bank of China,181.469000
9,China Merchants Bank,122.616000


<b>Question 1</b> Load the file <code>exchange_rates.csv</code> as a dataframe and find the exchange rate for British pounds with the symbol <code>GBP</code>, store it in the variable  <code>exchange_rate</code>, you will be asked for the number. Hint: set the parameter  <code>index_col</code> to 0.


In [6]:
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe
dataframe = extract_from_csv("exchange_rates.csv")
dataframe = pd.DataFrame(dataframe,columns=['Unnamed: 0','Rates'])
print(dataframe)

i=dataframe[dataframe['Unnamed: 0']=='GBP']
print(i)
exchange_rate = dataframe.iloc[9,1]
print(exchange_rate)





   Unnamed: 0         Rates
0         AUD      1.297088
1         BGN      1.608653
2         BRL      5.409196
3         CAD      1.271426
4         CHF      0.886083
5         CNY      6.483139
6         CZK     21.510117
7         DKK      6.119757
8         EUR      0.822504
9         GBP      0.732398
10        HKD      7.752509
11        HRK      6.222652
12        HUF    294.135549
13        IDR  14097.902616
14        ILS      3.281132
15        INR     73.011186
16        ISK    129.133081
17        JPY    103.791742
18        KRW   1105.839776
19        MXN     19.932966
20        MYR      4.043017
21        NOK      8.497121
22        NZD      1.393733
23        PHP     48.070406
24        PLN      3.732933
25        RON      4.008883
26        RUB     74.930827
27        SEK      8.292071
28        SGD      1.328261
29        THB     30.009870
30        TRY      7.418572
31        USD      1.000000
32        ZAR     15.118441
  Unnamed: 0     Rates
9        GBP  0.732398
0.

## Transform

Using <code>exchange_rate</code> and the `exchange_rates.csv` file find the exchange rate of USD to GBP. Write a transform function that

1.  Changes the `Market Cap (US$ Billion)` column from USD to GBP
2.  Rounds the Market Cap (US$ Billion)\` column to 3 decimal places
3.  Rename `Market Cap (US$ Billion)` to `Market Cap (GBP$ Billion)`


In [7]:
def extract_from_json(file_to_process):
    df = pd.read_json(file_to_process)
    return df
df=extract_from_json('bank_market_cap_1.json')
df = df.rename(columns={'Market Cap (US$ Billion)': 'Market Cap (GBP$ Billion)'})
def transform(file_to_process):
    return file_to_process*0.73239842
df['Market Cap (GBP$ Billion)'] = df['Market Cap (GBP$ Billion)'].apply(transform)
df=df.round({'Market Cap (GBP$ Billion)': 3})
df.style











,Name,Market Cap (GBP$ Billion)
0,JPMorgan Chase,286.319000
1,Industrial and Commercial Bank of China,252.834000
2,Bank of America,238.272000
3,Wells Fargo,225.588000
4,China Construction Bank,188.519000
5,HSBC Holdings PLC,160.593000
6,Agricultural Bank of China,148.856000
7,Citigroup Inc.,148.798000
8,Bank of China,132.908000
9,China Merchants Bank,89.804000


## Load

Create a function that takes a dataframe and load it to a csv named `bank_market_cap_gbp.csv`. Make sure to set `index` to `False`.


In [8]:
df.to_csv("bank_market_cap_gbp.csv", index=False)

def extract_from_csv(file_to_process):
    dfgbp = pd.read_csv(file_to_process)
    return dfgbp
dfgbp=extract_from_csv('bank_market_cap_gbp.csv')
dfgbp.style


    

,Name,Market Cap (GBP$ Billion)
0,JPMorgan Chase,286.319000
1,Industrial and Commercial Bank of China,252.834000
2,Bank of America,238.272000
3,Wells Fargo,225.588000
4,China Construction Bank,188.519000
5,HSBC Holdings PLC,160.593000
6,Agricultural Bank of China,148.856000
7,Citigroup Inc.,148.798000
8,Bank of China,132.908000
9,China Merchants Bank,89.804000


## Logging Function


Write the logging function <code>log</code> to log your data:


In [10]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M-%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open ("logfile.txt","a") as f:
        f.write (timestamp + ',' + message +'/n')
    

## Running the ETL Process


Log the process accordingly using the following <code>"ETL Job Started"</code> and <code>"Extract phase Started"</code>


In [12]:
# Write your code here
log ("ETL Job Started")
log ("Extract phase Started")

### Extract


<code>Question 2</code> Use the function <code>extract</code>, and print the first 5 rows, take a screen shot:


In [16]:
# Call the function here
df=extract_from_json('bank_market_cap_1.json')

# Print the rows here
df.head


<bound method NDFrame.head of                                        Name  Market Cap (US$ Billion)
0                            JPMorgan Chase                   390.934
1   Industrial and Commercial Bank of China                   345.214
2                           Bank of America                   325.331
3                               Wells Fargo                   308.013
4                   China Construction Bank                   257.399
..                                      ...                       ...
65                             Ping An Bank                    37.993
66                       Standard Chartered                    37.319
67                     United Overseas Bank                    35.128
68                                QNB Group                    33.560
69                              Bank Rakyat                    33.081

[70 rows x 2 columns]>

Log the data as <code>"Extract phase Ended"</code>


In [14]:
# Write your code here
log ("Extract phase Ended")

### Transform


Log the following  <code>"Transform phase Started"</code>


In [18]:
# Write your code here
log ("Transform phase Started")


<code>Question 3</code> Use the function <code>transform</code> and print the first 5 rows of the output, take a screen shot:


In [22]:
# Call the function here
df = df.rename(columns={'Market Cap (US$ Billion)': 'Market Cap (GBP$ Billion)'})
df['Market Cap (GBP$ Billion)'] = df['Market Cap (GBP$ Billion)'].apply(transform)
df=df.round({'Market Cap (GBP$ Billion)': 3})
# Print the first 5 rows here
df.head

<bound method NDFrame.head of                                        Name  Market Cap (GBP$ Billion)
0                            JPMorgan Chase                    286.319
1   Industrial and Commercial Bank of China                    252.834
2                           Bank of America                    238.272
3                               Wells Fargo                    225.588
4                   China Construction Bank                    188.519
..                                      ...                        ...
65                             Ping An Bank                     27.826
66                       Standard Chartered                     27.332
67                     United Overseas Bank                     25.728
68                                QNB Group                     24.579
69                              Bank Rakyat                     24.228

[70 rows x 2 columns]>

Log your data <code>"Transform phase Ended"</code>


In [ ]:
# Write your code here
log ("Transform phase Ended")


### Load


Log the following `"Load phase Started"`.


In [19]:
# Write your code here
log ("Load phase Started")

Call the load function


In [20]:
# Write your code here
df.to_csv("bank_market_cap_gbp.csv", index=False)

Log the following `"Load phase Ended"`.


In [21]:
# Write your code here
log ("Load phase Ended")


## Authors


Ramesh Sannareddy, Joseph Santrcangelo and Azim Hirjani


### Other Contributors


Rav Ahuja


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By        | Change Description                 |
| ----------------- | ------- | ----------------- | ---------------------------------- |
| 2020-11-25        | 0.1     | Ramesh Sannareddy | Created initial version of the lab |


Copyright © 2020 IBM Corporation. This notebook and its source code are released under the terms of the [MIT License](https://cognitiveclass.ai/mit-license?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork23455645-2022-01-01&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBM-DA0321EN-SkillsNetwork-21426264&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).
